In [1]:
using Distributed, LinearAlgebra
#rmprocs(workers())


In [2]:
nprocs()

1

In [3]:
1+1

2

# A Brief Feel for the Dream

In [4]:
# import Pkg; Pkg.add("CUDA")

In [5]:
using Distributed

if nprocs() == 1
  addprocs(4) 
end

# @everywhere begin
#     import Pkg
#     Pkg.activate($(Base.current_project()))
#   end

@everywhere using LinearAlgebra, DistributedArrays, CUDA
@everywhere import CUDA: @time
@everywhere CUDA.allowscalar(true)


## So you like linear algebra? <br>
<table style="width:100%">
  <tr>
    <th><img src="http://freecomputerbooks.com/covers/LAPACK-Users-Guide-Third-Edition.jpg" width="200"></th>
    <th><img src="http://www.netlib.org/scalapack/slug/slug_cover.gif"  width="200" style="background-color:white"></th>
      <th><img src="https://images-na.ssl-images-amazon.com/images/I/41uT31fSxBL._SX352_BO1,204,203,200_.jpg" width="200"></th>
    
  </tr>
    



## but you are innovating beyond the traditional libraries?

##  For demonstration let's make a matrix where you know more than the libraries know.

Let's create a custom  matrix structure that depends on n parameters, not $n^2$: <br> <br>
Diagonal(v) + v*v'
<br>

For example (n=3):
$$ \begin{pmatrix}
v_1 & & \\
& v_2 & \\
& & v_3
\end{pmatrix} + 
\begin{pmatrix}
v_1^2 & v_1 v_2 & v_1 v_3 \\
v_2 v_1 & v_2^2 & v_2 v_3 \\
v_3 v_1 & v_3 v_2 & v_3^2
\end{pmatrix} .
$$


# Build a Custom Type

In [6]:
struct MyMatrix{T, V<:AbstractVector{T}} <: AbstractMatrix{T}
   v::V
end

In [7]:
Base.size(A::MyMatrix)         =   length(A.v), length(A.v)
Base.getindex(A::MyMatrix,i,j) =   A.v[i]*(i==j)  + A.v[i]*A.v[j]


In [9]:
A = MyMatrix([1,10,100])

3×3 MyMatrix{Int64, Vector{Int64}}:
   2    10    100
  10   110   1000
 100  1000  10100

In [10]:
dump( A )  # n storage

MyMatrix{Int64, Vector{Int64}}
  v: Array{Int64}((3,)) [1, 10, 100]


In [11]:
dump(Matrix(A)) # n² storage

Array{Int64}((3, 3)) [2 10 100; 10 110 1000; 100 1000 10100]


## My very own largest eigensolver for my very own matrices

In [12]:
f(A::MyMatrix)  = λ ->  1 + mapreduce((v) -> v^2 / (v - λ)  , +, A.v)
f′(A::MyMatrix) = λ ->      mapreduce((v) -> v^2 / (v - λ)^2, +, A.v)

function LinearAlgebra.eigmax(A::MyMatrix; tol = eps(2.0), debug = false)
    x0 = maximum(A.v) + maximum(A.v)^2
    δ = f(A)(x0)/f′(A)(x0)
    while abs(δ) > x0 * tol               
        x0 -= δ
        δ = f(A)(x0)/f′(A)(x0) 
        debug && println("x = $x0, δ = $δ") # Debugging
    end
    x0
end

In [49]:
eigmax(A)

10200.107083707298

In [50]:
eigmax(Matrix(A))

10200.107083707298

## Go Heterogeneous

In [51]:
gpuA = MyMatrix(CuArray([1,2,3]))

3×3 MyMatrix{Int64, CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}}:
 2  2   3
 2  6   6
 3  6  12

In [52]:
distA = MyMatrix(distribute([1,2,3]))

3×3 MyMatrix{Int64, DArray{Int64, 1, Vector{Int64}}}:
 2  2   3
 2  6   6
 3  6  12

In [53]:
Matrix(gpuA)

3×3 Matrix{Int64}:
 2  2   3
 2  6   6
 3  6  12

In [54]:
Matrix(distA)

3×3 Matrix{Int64}:
 2  2   3
 2  6   6
 3  6  12

# Compare Timings

In [55]:
N = 4_000_000

4000000

In [56]:
v = randn(N)*.1

4000000-element Vector{Float64}:
 -0.16536603494134755
 -0.07095962323033238
  0.12860333463483845
  0.14645992984415482
  0.003357984162188212
 -0.07794335472007871
 -0.0015897862022342321
 -0.22725692828409097
 -0.21499830517389948
  0.1969989648104354
 -0.10064963054540912
 -0.03529032333885001
  0.037051935786844646
  ⋮
 -0.0976572985356512
 -0.0992330253327055
 -0.15163223376823298
  0.06491759158167733
  0.008797853688982669
 -0.08533607640600725
  0.004162485870364173
 -0.03289848558285172
 -0.0463076627186261
  0.14925475786646789
  0.19839469518880876
 -0.02305413448067844

In [57]:
dump(A)

MyMatrix{Int64, Vector{Int64}}
  v: Array{Int64}((3,)) [1, 10, 100]


In [58]:
A = MyMatrix(v)

4000000×4000000 MyMatrix{Float64, Vector{Float64}}:
 -0.13802       0.0117343    -0.0212666    …  -0.0328077     0.00381237
  0.0117343    -0.0659244    -0.00912564      -0.014078      0.00163591
 -0.0212666    -0.00912564    0.145142         0.0255142    -0.00296484
 -0.0242195    -0.0103927     0.0188352        0.0290569    -0.00337651
 -0.000555297  -0.000238281   0.000431848      0.000666206  -7.74154e-5
  0.0128892     0.00553083   -0.0100238    …  -0.0154635     0.00179692
  0.000262897   0.000112811  -0.000204452     -0.000315405   3.66511e-5
  0.0375806     0.0161261    -0.029226        -0.0450866     0.00523921
  0.0355534     0.0152562    -0.0276495       -0.0426545     0.0049566
 -0.0325769    -0.013979      0.0253347        0.0390835    -0.00454164
  0.016644      0.00714206   -0.0129439    …  -0.0199684     0.00232039
  0.00583582    0.00250419   -0.00453845      -0.00700141    0.000813588
 -0.00612713   -0.00262919    0.004765         0.00735091   -0.0008542
  ⋮          

In [59]:
@folded_threads eigmax(A) # using a macro

folded_context(true)  # context variable
eigmax(A)

eigmax( Threaded(A) ) # similar to CuArray and distribute (using the type mechanism to figure out to execute)

LoadError: LoadError: UndefVarError: `@folded_threads` not defined
in expression starting at In[59]:1

In [60]:
distA = MyMatrix(distribute(v));  

In [61]:
gpuA = MyMatrix(CuArray(v));

In [62]:
1+1

2

In [64]:
@time eigmax(distA) # run twice

  0.124375 seconds (15.55 k CPU allocations: 640.469 KiB)


39991.27661418578

In [66]:
CUDA.@timed(eigmax(gpuA)) # run twice

(value = 39991.2766141858, time = 0.193219635, cpu_bytes = 261712, cpu_gctime = 0.0, cpu_gcstats = Base.GC_Diff(261712, 0, 0, 4530, 0, 0, 0, 0, 0), gpu_bytes = 1920, gpu_memtime = 0.001664655999999997, gpu_memstats = (alloc_count = 96, alloc_bytes = 1920, free_count = 0, free_bytes = 0, total_time = 0.001664655999999997))

In [67]:
@time eigmax(gpuA)

  0.191692 seconds (4.52 k CPU allocations: 255.438 KiB) (96 GPU allocations: 1.875 KiB, 0.63% memmgmt time)


39991.2766141858

## Abstraction

1. A data structure is a mathematical matrix! <br>
2. A matrix can be serial, distributed, gpu <br>
BOTH ARE ABSTRACTIONS, BOTH use underlying similar mechanisms

## Why Julia?

1. Well designed abstractions <br>
2. Multiple dispatch <br>
3. Careful balance between static and dynamic <br>
4. Compiles to GPU at multiple levels, not just lowest CUDA level <br>
5. Metaprogramming across the stack <br>
6. Interfaces with LLVM <br>
7. Plays nicely with Python, legacy codes, ... <br>

<center> <a href="https://arxiv.org/abs/1411.1607">
Julia: A Fresh Approach to Numerical Computing 
</a> </center>

 <center> <img src="siam.png" width=400> </a> </center>